In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [19]:
import os
import glob
# path = './drive/My Drive/Colab Notebooks/Analysis/guten/sentences'
path = "./drive/My Drive/Colab Notebooks/Analysis/guten/sentences_react_predictions"

!cd './drive/My Drive/Colab Notebooks/Analysis/guten/sentences_react_predictions'; ls -U

all_files = glob.glob(os.path.join(path, "*.npy"))
print(len(all_files))

dct	  9411.npy   1401.npy	1905.npy   2480.npy   3330.npy	 492.npy
5343.npy  955.npy    1400.npy	1911.npy   24811.npy  3328.npy	 4925.npy
5402.npy  956.npy    14033.npy	19090.npy  24873.npy  33391.npy  48320.npy
541.npy   95.npy     1408.npy	19022.npy  24964.npy  3327.npy	 485.npy
540.npy   9371.npy   1404.npy	18947.npy  2488.npy   33435.npy  48225.npy
543.npy   946.npy    14070.npy	19188.npy  24923.npy  33516.npy  486.npy
5403.npy  9363.npy   14056.npy	19264.npy  24950.npy  33511.npy  4908.npy
5428.npy  940.npy    141.npy	19221.npy  25063.npy  33748.npy  50133.npy
551.npy   964.npy    142.npy	19229.npy  25016.npy  33504.npy  49819.npy
559.npy   9622.npy   14244.npy	19337.npy  2517.npy   33525.npy  50040.npy
55.npy	  9603.npy   1424.npy	1929.npy   2500.npy   33447.npy  4926.npy
552.npy   96.npy     14257.npy	19322.npy  25141.npy  33858.npy  500.npy
549.npy   9629.npy   14264.npy	1937.npy   25012.npy  33837.npy  498.npy
5500.npy  957.npy    1429.npy	1944.npy   25020.npy  33870.npy  49525

In [0]:
!pip install transformers

In [0]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [0]:
import keras.backend as K
import tensorflow as tf

from scipy.spatial.distance import jensenshannon
from numpy import asarray

kl_div = tf.keras.losses.KLDivergence()
 
# calculate the js divergence
def js_divergence(p, q):
	m = 0.5 * (p + q)
	return 0.5 * kl_div(p, m) + 0.5 * kl_div(q, m)

def js_distance(y_true, y_pred):
  return K.sqrt(js_divergence(y_true, y_pred))


Using TensorFlow backend.


In [0]:
from transformers import DistilBertPreTrainedModel, DistilBertModel
import torch.nn as nn
from transformers import BertTokenizer, DistilBertTokenizer


class MyDistilBertForSequenceClassificationReact(DistilBertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.distilbert = DistilBertModel(config)
        self.pre_classifier = nn.Linear(config.dim, config.dim)
        self.classifier = nn.Linear(config.dim, 5)
        # self.classifier = nn.Linear(config.dim, config.num_labels)
        self.dropout = nn.Dropout(config.seq_classif_dropout)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, head_mask=None, inputs_embeds=None, labels=None):
        
        distilbert_output = self.distilbert(
            input_ids=input_ids, attention_mask=attention_mask, head_mask=head_mask, inputs_embeds=inputs_embeds
        )
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        pooled_output = self.pre_classifier(pooled_output)  # (bs, dim)
        pooled_output = nn.ReLU()(pooled_output)  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        logits = self.classifier(pooled_output)  # (bs, dim)
        # new addition jordan
        logits = nn.ReLU()(logits)
        outputs = (logits,) + distilbert_output[1:]
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1, 5), labels.view(-1, 5))
            else:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

class MyDistilBertForSequenceClassificationExpress(DistilBertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.distilbert = DistilBertModel(config)
        self.pre_classifier = nn.Linear(config.dim, config.dim)
        self.classifier = nn.Linear(config.dim, 3)
        # self.classifier = nn.Linear(config.dim, config.num_labels)
        self.dropout = nn.Dropout(config.seq_classif_dropout)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, head_mask=None, inputs_embeds=None, labels=None):
        
        distilbert_output = self.distilbert(
            input_ids=input_ids, attention_mask=attention_mask, head_mask=head_mask, inputs_embeds=inputs_embeds
        )
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        pooled_output = self.pre_classifier(pooled_output)  # (bs, dim)
        pooled_output = nn.ReLU()(pooled_output)  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        logits = self.classifier(pooled_output)  # (bs, dim)
        # new addition jordan
        logits = nn.ReLU()(logits)
        outputs = (logits,) + distilbert_output[1:]
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1, 3), labels.view(-1, 3))
            else:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [0]:
def calc_max_len(sentences):
  max_len = 75
  # # For every sentence...
  # for sent in sentences:
  #     # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
  #     input_ids = tokenizer.encode(sent, add_special_tokens=True)

  #     # Update the maximum sentence length.
  #     max_len = max(max_len, len(input_ids))

  # max_len = min(max_len, 512)
  # print()
  # print('Max sentence length: ', max_len)
  return max_len

In [0]:
import pandas as pd
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

def predict_sentences(model, tokenizer, sentences):
  max_len = calc_max_len(sentences)

  input_ids = []
  attention_masks = []

  for sent in sentences:
      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = max_len,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      input_ids.append(encoded_dict['input_ids'])
      attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  batch_size = 32
  prediction_data = TensorDataset(input_ids, attention_masks)
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

  # Prediction on test set
  print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))
  # Put model in evaluation mode
  model.eval()
  # Tracking variables 
  predictions = []
  # Predict 
  for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)
    
    b_input_ids, b_input_mask = batch
    
    with torch.no_grad():
        outputs = model(b_input_ids, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    predictions.append(logits)
  return predictions

# Express

Load express base model

In [0]:
# Load a trained model and vocabulary that you have fine-tuned
output_dir = "./drive/My Drive/Colab Notebooks/MainModels/express_base_model"
model = MyDistilBertForSequenceClassificationExpress.from_pretrained(
    output_dir, # Use the 12-layer BERT model, with an uncased vocab.
    # num_labels = 5, # The number of output labels--2 for binary classification.
    num_labels = 1, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda()
tokenizer = DistilBertTokenizer.from_pretrained(output_dir,do_lower_case=True)


Express base model predictions

In [0]:
import os
import glob
import numpy as np
import time

def get_names(files):
  names = []
  for f in files:
    base_file = os.path.basename(f)
    new_file = os.path.splitext(base_file)[0]
    names.append(new_file)
  return names

path = './drive/My Drive/Colab Notebooks/Analysis/scripts/sentences'
out_path = './drive/My Drive/Colab Notebooks/Analysis/scripts/sentences_express_predictions/'

all_files = glob.glob(os.path.join(path, "*.txt"))
print('all count: %s' % len(all_files))
all_names = get_names(all_files)
print(all_names[:10])

completed_files = glob.glob(os.path.join(out_path, "*.npy"))
print('completed count: %s' % len(completed_files))
completed_names = get_names(completed_files)
print(completed_names[:10])

curr_names = list(set(all_names) - set(completed_names))
curr_files = [path + '/' + name + '.txt' for name in curr_names]
print('curr count: %s' % len(curr_files))


t0 = time.time()
for i, filename in enumerate(curr_files):

  t1 = time.time()

  with open(filename, "r", encoding='utf-8') as f:
    try:
      if i % 10 == 0:
        print("Processing file:{} index:{} ({}%)".format(os.path.basename(filename), i, 100*i//len(curr_files)), end="")

      sentences = f.read().splitlines()
      predictions = predict_sentences(model, tokenizer, sentences)
      predictions = np.array(predictions)
      predictions = np.vstack(predictions)
      base_file = os.path.basename(filename)
      new_file = os.path.splitext(base_file)[0] + '.npy'
      new_path = out_path + new_file

      np.save(new_path, predictions, allow_pickle=True)
    except Exception as e:
      print('file err: %s' % e)
      continue


  t2 = time.time()
  print(" {}s (total: {}s)".format(t2-t1, t2-t0))
  print()

all count: 1124
['Back-up-Plan,-The', 'Bachelor-Party', 'Bad-Boys', 'Babe', 'Bad-Country', 'Backdraf', 'Basic', 'Barry-Lyndon', 'Batman-2', 'Batman']
completed count: 579
['Back-up-Plan,-The', 'Bachelor-Party', 'Bad-Boys', 'Babe', 'Bad-Country', 'Backdraf', 'Basic', 'Barry-Lyndon', 'Batman-2', 'Batman']
curr count: 545
Processing file:Men-Who-Stare-at-Goats,-The.txt index:0 (0%)Predicting labels for 2,684 test sentences...
 12.601267576217651s (total: 12.601553678512573s)

Predicting labels for 3,826 test sentences...
 17.332268953323364s (total: 29.934192657470703s)

Predicting labels for 2,293 test sentences...
 10.168763160705566s (total: 40.103545904159546s)

Predicting labels for 2,944 test sentences...
 13.693200588226318s (total: 53.797882080078125s)

Predicting labels for 1,737 test sentences...
 7.95036506652832s (total: 61.748626470565796s)

Predicting labels for 3,571 test sentences...
 16.643027782440186s (total: 78.3927550315857s)

Predicting labels for 2,843 test sentence

# React

Load react base model


In [0]:
# Load a trained model and vocabulary that you have fine-tuned
output_dir = "./drive/My Drive/Colab Notebooks/MainModels/react_base_model"
model = MyDistilBertForSequenceClassificationReact.from_pretrained(
    output_dir, # Use the 12-layer BERT model, with an uncased vocab.
    # num_labels = 5, # The number of output labels--2 for binary classification.
    num_labels = 1, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda()
tokenizer = DistilBertTokenizer.from_pretrained(output_dir,do_lower_case=True)


React base model predictions

In [0]:
import os
import glob
import numpy as np
import time

path = './drive/My Drive/Colab Notebooks/Analysis/sentences'
out_path = './drive/My Drive/Colab Notebooks/Analysis/sentences_react_predictions/'
all_files = glob.glob(os.path.join(path, "*.txt"))

t0 = time.time()
for i, filename in enumerate(all_files[1003:]):
  t1 = time.time()
  if i % 10 == 0:
    print("Processing file:{} index:{} ({}%)".format(os.path.basename(filename), i, 100*i//len(all_files)), end="")


  with open(filename, "r", encoding='utf-8') as f:
    sentences = f.read().splitlines()
    predictions = predict_sentences(model, tokenizer, sentences)
    predictions = np.array(predictions)
    predictions = np.vstack(predictions)

  base_file = os.path.basename(filename)
  new_file = os.path.splitext(base_file)[0] + '.npy'
  new_path = out_path + new_file

  np.save(new_path, predictions, allow_pickle=True)

  t2 = time.time()
  print(" {}s (total: {}s)".format(t2-t1, t2-t0))
  print()

Processing file:10007.txt index:0 (0%)Predicting labels for 1,436 test sentences...
 7.506017208099365s (total: 7.506219387054443s)

Predicting labels for 4,149 test sentences...
 22.251609086990356s (total: 29.758013486862183s)

Predicting labels for 4,409 test sentences...
 21.943938970565796s (total: 51.7027542591095s)

Predicting labels for 4,992 test sentences...
 25.706188201904297s (total: 77.4110312461853s)

Predicting labels for 3,780 test sentences...
 19.596242904663086s (total: 97.00867033004761s)

Predicting labels for 3,060 test sentences...
 15.797215938568115s (total: 112.80679440498352s)

Predicting labels for 1,385 test sentences...
 7.24506950378418s (total: 120.05337309837341s)

Predicting labels for 1,463 test sentences...
 7.745054006576538s (total: 127.79956412315369s)

Predicting labels for 2,870 test sentences...
 14.66627025604248s (total: 142.46693587303162s)

Predicting labels for 6,477 test sentences...
 32.68364357948303s (total: 175.15124988555908s)

Proc

# Meta

Combine

In [0]:
react_predictions = np.vstack(react_predictions)
express_predictions = np.vstack(express_predictions)
print(react_predictions.shape)
print(express_predictions.shape)
data = np.concatenate((react_predictions, express_predictions),axis=1)
print(data.shape)

(840, 5)
(840, 3)
(840, 8)


Meta Learner

In [0]:
from keras.models import load_model
output_dir = "./drive/My Drive/Colab Notebooks/MainModels/"
dependencies = {
    'js_distance': js_distance
}
model = load_model(output_dir + 'ensemble_meta_learner.h5', dependencies)
predictions = model.predict(data)

def map_affect_reaction_to_index(reaction):
  reactions = {"joy": 0, "surprise": 1, "sadness": 2, "anger": 3}
  return reactions[reaction]

def map_fb_idx_2_affect_idx(idx):
  # love_count,wow_count,haha_count,sad_count,angry_count
  if idx == 0 or idx == 2:
    # joy
    return 0
  elif idx == 1:
    # surprise
    return 1
  elif idx == 3:
    # sadness
    return 2
  elif idx == 4:
    # anger
    return 3


labels_idx = list(map(lambda react: map_affect_reaction_to_index(react), labels))
# print(labels_idx[4])
# print(predictions[4])
predictions_argmax = list(map(lambda pred: np.argmax(pred), predictions))
# print(predictions_argmax[4])
predictions_idx = list(map(lambda idx: map_fb_idx_2_affect_idx(idx), predictions_argmax))

names = ["joy", "surprise", "sadness", "anger"]
print(sentences[800])
print(names)
print(labels_idx[800])
print(predictions_idx[800])
from sklearn import metrics

print(metrics.classification_report(labels_idx, predictions_idx, target_names=names))


ex border patrol agent beaten in prison
['joy', 'surprise', 'sadness', 'anger']
2
2
              precision    recall  f1-score   support

         joy       0.60      0.87      0.71       362
    surprise       0.34      0.16      0.22       184
     sadness       0.81      0.50      0.61       202
       anger       0.46      0.55      0.50        92

    accuracy                           0.59       840
   macro avg       0.55      0.52      0.51       840
weighted avg       0.58      0.59      0.56       840



Manual test

In [0]:
affective_predictions = model.predict(data)